<a href="https://colab.research.google.com/github/hiteshJindal/Python_Projects/blob/master/Audio_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

class VivaeAudioDataset:
    def __init__(self, audio_folder_path, sr=22050, n_mels=128, hop_length=512, n_fft=2048):
        self.audio_folder_path = audio_folder_path
        self.sr = sr
        self.n_mels = n_mels
        self.hop_length = hop_length
        self.n_fft = n_fft

    def load_dataset(self):
        audio_data = []
        labels = []
        for root, dirs, files in os.walk(self.audio_folder_path):
            for file in files:
                if file.endswith('.wav'):
                    audio_path = os.path.join(root, file)
                    label = root.split('/')[-1]
                    labels.append(label)
                    audio, _ = librosa.load(audio_path, sr=self.sr)
                    spectrogram = librosa.feature.melspectrogram(audio, sr=self.sr, n_mels=self.n_mels,
                                                                 hop_length=self.hop_length, n_fft=self.n_fft)
                    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
                    audio_data.append(log_spectrogram)
        le = LabelEncoder()
        labels = le.fit_transform(labels)
        labels = to_categorical(labels, num_classes=len(set(labels)))
        return np.array(audio_data), labels


In [71]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [17]:
dataset = VIVAEAudioDataset("/gdrive/MyDrive/VIVAE/full_set")
train_spectrograms, train_labels, test_spectrograms, test_labels = dataset.get_train_data()
# Train your machine learning model using the spectrograms and labels

<ipython-input-16-641f87b624ff>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_spectrograms.T = np.array([spectrogram for spectrogram in train_spectrograms])


ValueError: ignored

In [25]:
# Instantiate the dataset class
dataset = VivaeAudioDataset(audio_folder_path='/gdrive/MyDrive/VIVAE/full_set')

# Load the dataset
audio_data, labels = dataset.load_dataset()

# Split the dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(audio_data, labels, test_size=0.2, random_state=42)

# Build the model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(set(labels)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


<ipython-input-24-e35acfcf00f8>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(audio_data), labels


ValueError: ignored

In [26]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.utils import shuffle

class VivaeAudioDataset:
    def __init__(self, audio_folder_path, sr=22050, n_mels=128, hop_length=512, n_fft=2048):
        self.audio_folder_path = audio_folder_path
        self.sr = sr
        self.n_mels = n_mels
        self.hop_length = hop_length
        self.n_fft = n_fft

    def load_dataset(self):
        audio_data = []
        labels = []
        for root, dirs, files in os.walk(self.audio_folder_path):
            for file in files:
                if file.endswith('.wav'):
                    audio_path = os.path.join(root, file)
                    label = root.split('/')[-1]
                    labels.append(label)
                    audio, _ = librosa.load(audio_path, sr=self.sr)
                    spectrogram = librosa.feature.melspectrogram(audio, sr=self.sr, n_mels=self.n_mels,
                                                                 hop_length=self.hop_length, n_fft=self.n_fft)
                    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
                    audio_data.append(log_spectrogram)
        le = LabelEncoder()
        labels = le.fit_transform(labels)
        labels = to_categorical(labels, num_classes=len(set(labels)))
        # shuffle the dataset
        audio_data, labels = shuffle(audio_data, labels, random_state=42)
        return np.array(audio_data), labels


In [27]:
# Instantiate the dataset class
dataset = VivaeAudioDataset(audio_folder_path='/gdrive/MyDrive/VIVAE/full_set')

# Load the dataset
audio_data, labels = dataset.load_dataset()

# Split the dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(audio_data, labels, test_size=0.2, random_state=42)

# Build the model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(set(labels)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


<ipython-input-26-7bd0ef0cd2d6>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(audio_data), labels


ValueError: ignored

In [12]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.utils import shuffle

class VivaeAudioDataset:
    def __init__(self, audio_folder_path, sr=22050, n_mels=128, hop_length=512, n_fft=2048):
        self.audio_folder_path = audio_folder_path
        self.sr = sr
        self.n_mels = n_mels
        self.hop_length = hop_length
        self.n_fft = n_fft

    def load_dataset(self):
        audio_data = []
        labels = []
        for root, dirs, files in os.walk(self.audio_folder_path):
            for file in files:
                if file.endswith('.wav'):
                    audio_path = os.path.join(root, file)
                    label = root.split('/')[-1]
                    labels.append(label)
                    audio, _ = librosa.load(audio_path, sr=self.sr)
                    spectrogram = librosa.feature.melspectrogram(audio, sr=self.sr, n_mels=self.n_mels,
                                                                 hop_length=self.hop_length, n_fft=self.n_fft)
                    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
                    audio_data.append(log_spectrogram)
        le = LabelEncoder()
        labels = le.fit_transform(labels)
        print("Unique labels:", np.unique(labels))
        labels = to_categorical(labels, num_classes=len(set(labels)))
        # shuffle the dataset
        audio_data, labels = shuffle(audio_data, labels, random_state=42)
        # reshape the input data to (num_samples, 128, 128, 1)
        audio_data = np.expand_dims(audio_data, axis=-1)
        audio_data = np.repeat(audio_data, 3, axis=-1)
        audio_data = np.transpose(audio_data, (0, 2, 1, 3))
        return audio_data, labels


In [13]:
# Instantiate the dataset class
dataset = VivaeAudioDataset(audio_folder_path='/gdrive/MyDrive/VIVAE/full_set')

# Load the dataset
audio_data, labels = dataset.load_dataset()

# Split the dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(audio_data, labels, test_size=0.2, random_state=42)


# Build the model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(set(labels)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Unique labels: [0]


/usr/local/lib/python3.8/dist-packages/numpy/lib/shape_base.py:591: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = asanyarray(a)


ValueError: ignored

In [14]:
import os
import librosa
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

class AudioDataset:
    def __init__(self, audio_folder_path, n_mels=128, n_fft=2048, hop_length=512, sr=22050):
        self.audio_folder_path = audio_folder_path
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.sr = sr

    def load_dataset(self):
        audio_data = []
        labels = []
        for root, dirs, files in os.walk(self.audio_folder_path):
            for file in files:
                if file.endswith('.wav'):
                    audio_path = os.path.join(root, file)
                    label = audio_path.split('_')[2]
                    labels.append(label)
                    audio, _ = librosa.load(audio_path, sr=self.sr)
                    spectrogram = librosa.feature.melspectrogram(audio, sr=self.sr, n_mels=self.n_mels,
                                                                 hop_length=self.hop_length, n_fft=self.n_fft)
                    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
                    audio_data.append(log_spectrogram)
        le = LabelEncoder()
        labels = le.fit_transform(labels)
        labels = to_categorical(labels, num_classes=len(set(labels)))

        # shuffle the dataset
        audio_data = np.array(audio_data)
        audio_data = audio_data.reshape(-1,1)        
        
        # reshape the label array to (num_samples, 1)
        labels = labels.reshape(-1, 1)
        
        # split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(audio_data, labels, test_size=0.2, random_state=42)
        return X_train, X_test, y_train, y_test


In [86]:
!pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10216 sha256=0187f4f64429dd6720e9ef60fc5cc4def5f9b96fb2451a28864e6c3559390f26
  Stored in directory: /root/.cache/pip/wheels/51/e4/5c/d3610ed4476515e540ff4096e3e9a8a3e701dedfe1072eb000
Successfully built pad_sequences


In [87]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# set the path to the audio dataset folder
audio_folder_path = '/gdrive/MyDrive/VIVAE/full_set'

# create an instance of the AudioDataset class
audio_dataset = AudioDataset(audio_folder_path)

# load the dataset and split it into training and testing sets
X_train, X_test, y_train, y_test = audio_dataset.load_dataset()
print(X_train)
# define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model

model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_data=(X_test_padded, y_test))


NameError: ignored

In [15]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split

class AudioData:
    def __init__(self, data_dir, test_size=0.2, sr=22050, n_mels=128, n_fft=2048, hop_length=512):
        self.data_dir = data_dir
        self.test_size = test_size
        self.sr = sr
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.class_names = None
        
    def load_data(self):
        # Get all file paths and labels
        file_paths = []
        labels = []
        for i, (dirpath, dirnames, filenames) in enumerate(os.walk(self.data_dir)):
            if dirpath != self.data_dir:
                class_name = dirpath.split("_")[2]
                for file in filenames:
                    file_path = os.path.join(dirpath, file)
                    file_paths.append(file_path)
                    labels.append(class_name)
                    print(class_name)
        self.class_names = np.unique(labels)
        
        # Convert audio files to Mel spectrograms
        mel_spectrograms = []
        for file_path in file_paths:
            # Load audio file
            audio, _ = librosa.load(file_path, sr=self.sr)
            # Create Mel spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(audio, sr=self.sr, n_mels=self.n_mels,
                                                             n_fft=self.n_fft, hop_length=self.hop_length)
            mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
            mel_spectrograms.append(mel_spectrogram)
        self.X = np.array(mel_spectrograms)
        
        # Create labels for each audio file
        label_dict = {class_name: i for i, class_name in enumerate(self.class_names)}
        self.y = np.array([label_dict[label] for label in labels])
        
        # Split data into training and testing sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=self.test_size, random_state=42)
        
        # Print data shapes
        print("X_train shape: ", self.X_train.shape)
        print("X_test shape: ", self.X_test.shape)
        print("y_train shape: ", self.y_train.shape)
        print("y_test shape: ", self.y_test.shape)


In [16]:

audio_folder_path = '/gdrive/MyDrive/VIVAE/full_set'

# create an instance of the AudioDataset class
audio_dataset = AudioData(audio_folder_path)
audio_dataset.load_data()

ValueError: ignored

In [39]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split

class AudioDataLoader:
    def __init__(self, data_dir, sample_rate=22050, n_mels=128, n_fft=2048, hop_length=512):
        self.data_dir = data_dir
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.train_data = None
        self.test_data = None
    
    def load_data(self):
        X = []
        y = []
        for filename in os.listdir(self.data_dir):
            if filename.endswith(".wav"):
                filepath = os.path.join(self.data_dir, filename)
                label = filename.split("_")[2]  # Extract label from filename
                y.append(label)
                print(label)
                X.append(self.process_audio(filepath))
        X = np.array(X)
        y = np.array(y)
        return X, y
    
    def process_audio(self, filepath):
        signal, sr = librosa.load(filepath, sr=self.sample_rate)
        mel_spec = librosa.feature.melspectrogram(signal, sr=sr, n_mels=self.n_mels,
                                                  n_fft=self.n_fft, hop_length=self.hop_length)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        return mel_spec_db.reshape(self.n_mels, -1, 1)  # Add channel dimension
    
    def split_data(self, X, y, test_size=0.2, random_state=42):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        self.train_data = (X_train, y_train)
        self.test_data = (X_test, y_test)
        return X_train, X_test, y_train, y_test

    def fit_model(self, model):
        X_train, y_train = self.train_data
        X_test, y_test = self.test_data
        # Fit the model on the training data
        model.fit(X_train, y_train)
        # Evaluate the model on the test data
        test_loss, test_acc = model.evaluate(X_test, y_test)
        print('Test loss:', test_loss)
        print('Test accuracy:', test_acc)


In [6]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split

class AudioDataLoader:
    def __init__(self, data_dir, sample_rate=22050, n_mels=128, n_fft=2048, hop_length=512):
        self.data_dir = data_dir
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.train_data = None
        self.test_data = None
    
    def load_data(self):
        X = []
        y = []
        max_frames = 0
        for filename in os.listdir(self.data_dir):
            if filename.endswith(".wav"):
                filepath = os.path.join(self.data_dir, filename)
                label = filename.split("_")[1]  # Extract label from filename
                y.append(label)
                mel_spec = self.process_audio(filepath)
                max_frames = max(max_frames, mel_spec.shape[1])
                X.append(mel_spec)
        X = self.pad_spectrograms(X, max_frames)
        y = np.array(y)
        unique_labels, counts = np.unique(y, return_counts=True)
        print("Number of unique classes:", len(unique_labels))
        return X, y
    
    def process_audio(self, filepath):
        signal, sr = librosa.load(filepath, sr=self.sample_rate)
        mel_spec = librosa.feature.melspectrogram(signal, sr=sr, n_mels=self.n_mels,
                                                  n_fft=self.n_fft, hop_length=self.hop_length)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        return mel_spec_db
    
    def pad_spectrograms(self, X, max_frames):
        X_padded = []
        for mel_spec in X:
            n_frames = mel_spec.shape[1]
            pad_width = ((0, 0), (0, max_frames - n_frames))
            mel_spec_padded = np.pad(mel_spec, pad_width=pad_width, mode='constant')
            X_padded.append(mel_spec_padded)
        return np.array(X_padded)
    
    def split_data(self, X, y, test_size=0.2, random_state=42):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        self.train_data = (X_train, y_train)
        self.test_data = (X_test, y_test)
        return X_train, X_test, y_train, y_test

    def fit_model(self, model):
        X_train, y_train = self.train_data
        X_test, y_test = self.test_data
        # Fit the model on the training data
        model.fit(X_train, y_train)
        # Evaluate the model on the test data
        test_loss, test_acc = model.evaluate(X_test, y_test)
        print('Test loss:', test_loss)
        print('Test accuracy:', test_acc)


In [23]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
model = Sequential([
    Dense(512, activation='relu', input_shape=(loader.n_mels, 87)),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(6, activation='softmax'),
    Flatten()  # Add this layer
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))



Epoch 1/10


ValueError: ignored

In [19]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [17]:
print(X_train)

[[-43.928017 -42.581963 -40.912254 ...   0.         0.         0.      ]
 [-53.354927 -55.658234 -56.1817   ...   0.         0.         0.      ]
 [-65.724724 -67.39293  -66.91714  ...   0.         0.         0.      ]
 ...
 [-76.3805   -72.36535  -68.82615  ...   0.         0.         0.      ]
 [-80.       -80.       -80.       ...   0.         0.         0.      ]
 [-80.       -80.       -80.       ...   0.         0.         0.      ]]


In [7]:
audio_folder_path = '/gdrive/MyDrive/VIVAE/full_set'
loader = AudioDataLoader(data_dir=audio_folder_path)

X, y = loader.load_data()
X_train, X_test, y_train, y_test = loader.split_data(X, y)


Number of unique classes: 6


In [49]:
print(model.output_shape)
print(y_test.shape)

(None, 6)
(221, 6)


In [48]:
loader.fit_model(model)


ValueError: ignored

In [80]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

class VIVAEAudioDataset:
    def __init__(self, data_path, n_mels=128, n_fft=2048, hop_length=512):
        self.data_path = data_path
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.sr = 44100
        self.labels = []
        self.le = LabelEncoder()
        self.X = []
        self.y = []
    
    def load_data(self):
        audio_files = os.listdir(self.data_path)
        for file in audio_files:
            filepath = os.path.join(self.data_path, file)
            if os.path.isfile(filepath) and file.endswith('.wav'):
                # Load audio file
                audio, sr = librosa.load(filepath, sr=self.sr)
                # Extract mel spectrogram
                mel_spec = librosa.feature.melspectrogram(y=audio, sr=self.sr, n_mels=self.n_mels, n_fft=self.n_fft, hop_length=self.hop_length)
                mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
                # Create label from filename
                label = file.split('_')[1]
                self.labels.append(label)
                # Append mel spectrogram and label to X and y
                self.X.append(mel_spec_db)
                self.y.append(label)
        
        # Encode labels
        self.le.fit(self.labels)
        self.y = self.le.transform(self.y)
        self.y = to_categorical(self.y, 6)
        
        
        # Split dataset
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
    
    def fit_model(self, model):
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(self.X_train, self.y_train, epochs=10, batch_size=128, validation_data=(self.X_test, self.y_test))


In [81]:
audio_folder_path = '/gdrive/MyDrive/VIVAE/full_set'
dataset = VIVAEAudioDataset(audio_folder_path)
dataset.load_data()

In [63]:
print(model.output_shape) 
print(y_train.shape)
model.summary()

(None, 6)
(884, 6)
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 126, 85, 32)       320       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 63, 42, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 61, 40, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 30, 20, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 28, 18, 128)       73856     
                                                                 
 max_pooling2d_26 (MaxPoolin  (Non

In [83]:
# Build the model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train[0].shape+(1,)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(6, activation='relu'))


# Train the model
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [89]:
import pad_sequences

X_train_padded = pad_sequences(X_train, maxlen=128, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=128, padding='post', truncating='post')

model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_data=(X_test_padded, y_test))


TypeError: ignored

In [84]:
dataset.fit_model(model)

ValueError: ignored